## TensorFlow计算图（Graph）与张量（Tensor）

选择环境：Anaconda Python 3.5.2  
安装Tensorflow：Python 3.5环境下运行pip install --upgrade --ignore-installed tensorflow  
参考书籍：《TensorFlow实战Google深度学习框架（第2版）》

In [1]:
#测试
import tensorflow as tf;
a = tf.constant([1.0,2.0],name="a")
b = tf.constant([2.0,3.0],name="b")
result = a + b;
sess = tf.Session()
print(sess.run(result))

[3. 5.]


### 3.1 计算图

Tensorflow程序第一阶段为定义计算图中所有的计算，第二阶段为执行计算。  
这里介绍计算定义阶段。

In [2]:
import tensorflow as tf
a = tf.constant([1.0,2.0],name="a") #定义常量
b = tf.constant([2.0,3.0],name="b")
result = a + b #张量相加

tf.get_default_graph() 获取当前默认的计算图  
a.graph() 查看张量所属计算图  
tf.Graph() 生成新的计算图，不同计算图张量和运算不共享

In [3]:
print(a.graph is tf.get_default_graph()) #未指定，默认计算图

True


In [4]:
#示意在不同计算图上定义和使用变量
import tensorflow as tf

g1 = tf.Graph()
with g1.as_default():
    # 在计算图 g1 中定义变量“v”,并设置初始值为 0
    v = tf.get_variable("v", shape=[1], initializer=tf.zeros_initializer)

g2 = tf.Graph()
with g2.as_default():
    # 在计算图 g2 中定义变量“v”设置初始值为 1
    v = tf.get_variable("v", shape=[1], initializer=tf.ones_initializer)

# 在计算图 g1 中，读取变量‘v’的值
with tf.Session(graph=g1) as sess:
    tf.global_variables_initializer().run() #初始化全局变量
    with tf.variable_scope("", reuse=True):
        # 在计算图 g1 中，变量 ‘v’的值取值应该为 0，所以下面这行会输出[0.]
        print(sess.run(tf.get_variable("v")))

# 在计算图 g2 中，读取变量‘v’的值
with tf.Session(graph=g2) as sess:
    tf.global_variables_initializer().run() #初始化全局变量
    with tf.variable_scope("",reuse=True):
        # 在计算图 g2 中，变量 ‘v’的值取值应该为 1，所以下面这行会输出[1.]
        print(sess.run(tf.get_variable("v")))

[0.]
[1.]


tf.Graph.device() 指定运行计算的设备，为TensorFlow使用GPU提供了机制

In [5]:
a = tf.constant([1.0,2.0],name="a")
b = tf.constant([2.0,3.0],name="b")
g = tf.Graph()
#指定计算运行的设备
with g.device('/gpu:0'):
    result = a + b
    sess = tf.Session()
    print(sess.run(result))

[3. 5.]


### 3.2 TensorFlow数据模型——张量 

在TensorFlow程序中，所有数据都通过张量的形式来表示。  
张量在TensorFlow中的实现并不是直接采用数组的形式，只是对TensorFlow中运算结果的引用。在张量中并没有真正保存数字，它保存的是如何得到这些数字的计算过程。

In [6]:
import tensorflow as tf
#tf.constant是一个计算，这个计算的结果为一个张量，保存在变量a中
a = tf.constant([1.0,2.0],name="a")
b = tf.constant([2.0,3.0],name="b")
result = tf.add(a,b,name="add")
print(result)

Tensor("add_3:0", shape=(2,), dtype=float32)


计算结果不是一个具体的数字，而是一个张量的结构：名字name、维度shape、类型type。  
名字是张量的唯一标识符，给出了这个张量是如何计算出来的。  
node:src_output中node为节点的名称，src_output为当前张量来自节点的第几个输出。  
上例中result这个张量是计算节点add输出的第一个结果（编号从0开始）。

TensorFlow会检查参与运算的张量的类型，类型不匹配时报错：

In [7]:
a = tf.constant([1.0, 2.0], name="a") #默认类型为float32
b = tf.constant([2.0, 3.4], name="b")
c = tf.constant([4, 5], name="c") #默认类型为int32
d = tf.constant([4, 5], name="d", dtype=tf.float32) #建议使用dtype明确类型
result = a + b

#类型不匹配 会报错
#result1 = a + c

#指定类型 dtype=tf.float32 正常运行
result2 = a + d

tf支持的数据类型:  
实数(tf.float32,tf.float64)  
整数(tf.int8,tf.int16,tf.int32,tf.int64,tf.uint8)  
布尔型(tf.bool)  
复数(tf.complex64,tf.complex128)

张量使用主要可以总结为两大类。  
第一类是对中间计算结果的引用，当一个计算包含很多中间结果时，使用张量可以大大提高代码的可读性。在卷积神经网络中，卷积层或者池化层有可能改变张量的维度，通过result.get_shape来获取结果张量的维度信息可以免去人工计算的麻烦；  
第二类是计算图构造完成后，张量可以用来获得计算机结果，也就是得到真实的数字。张量本身没有存储具体的数字，但是通过会话(session)就可以得到这些具体的数字。比如使用tf.Session().run(result)